In [4]:
import pandas as pd
import numpy as np
from graph_traffic.config import data_path

import matplotlib as mpl
import os

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

### Locations data

In [6]:
locs = pd.read_csv(f'{data_path}/01-raw/meteo/Estaciones_control_datos_meteorologicos.csv', sep=';', encoding="cp1252", usecols=['CÓDIGO_CORTO', 'LONGITUD', 'LATITUD'])\
    .dropna(subset=['CÓDIGO_CORTO'])
locs.columns = ['codigo_corto', 'longitud', 'latitud']
locs.to_csv(f'{data_path}/02-by-location-month/estaciones.csv', index=False)
locs.head()

,codigo_corto,longitud,latitud
0,4.0,-3.712257,40.423882
1,8.0,-3.682316,40.421553
2,16.0,-3.639242,40.440046
3,18.0,-3.731836,40.394782
4,24.0,-3.747345,40.419358


In [9]:
cods_cortos = sorted(locs.codigo_corto.values)
cods_cortos

[4.0,
 8.0,
 16.0,
 18.0,
 24.0,
 35.0,
 36.0,
 38.0,
 39.0,
 54.0,
 56.0,
 58.0,
 59.0,
 102.0,
 103.0,
 104.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0]

#### Historical data

In [8]:
month_dict = {
    1: "ene", 2: "feb", 3: "mar", 4: "abr", 5: "may", 6: "jun", 7: "jul", 8: "ago", 9: "sep", 10: "oct", 11: "nov", 12: "dic"
}
hcols = [f"H{d:02}" for d in range(1, 25)]
vcols = [f"V{d:02}" for d in range(1, 25)]

years = sorted(list(set([file_name.split(".")[0][-2:] for file_name in os.listdir(f"{data_path}/01-raw/meteo/history")])))
years

['19', '20', '21']

In [16]:
files = []
for year in years:
    files += [f"{data_path}/01-raw/meteo/history/{month_dict[i]}_meteo{year}.csv" for i in range(1, 13)]
n_files = len(files)
for i, file in enumerate(files):
    print(f"{i+1}/{n_files}", end="\r")
    df = pd.read_csv(file, sep=';')
    df['fecha'] = pd.to_datetime(dict(year=df.ANO, month=df.MES, day=df.DIA))
    magnitudes = {
        80: 'radiacion_ultravioleta',
        81: 'velocidad_viento',
        82: 'dir_viento',
        83: 'temperatura',
        86: 'humedad_relativa',
        87: 'presion_barometrica',
        88: 'radiacion_solar',
        89: 'precipitacion'
    }
    df['magnitud'] = df.MAGNITUD.apply(lambda x: magnitudes[x])
    df = df[df.magnitud!='radiacion_ultravioleta']
    del df['PROVINCIA'], df['MUNICIPIO'], df['PUNTO_MUESTREO'], df['ANO'], df['MES'], df['DIA'], df['MAGNITUD']
    df = df.rename(columns={'ESTACION': 'estacion'})
    fechas = sorted(df.fecha.unique())

    magnitudes = ['velocidad_viento', 'dir_viento', 'temperatura', 'humedad_relativa', 'presion_barometrica', 'radiacion_solar', 'precipitacion']
    magnitudes_x = [f'{m}_x' for m in magnitudes]
    magnitudes_y = [f'{m}_y' for m in magnitudes]
    dfn = pd.DataFrame([], columns=['estacion', 'fecha', 'hora'] + magnitudes_x + magnitudes_y)

    for estacion in cods_cortos:
        for fecha in fechas:
            dff = df[(df.fecha==fecha) & (df.estacion==estacion)]

            dfh = dff[['magnitud']+hcols].set_index('magnitud').transpose()
            dfh['hora'] = dfh.index.to_series().apply(lambda x: int(x[1:]))

            dfv = dff[['magnitud']+vcols].set_index('magnitud').transpose()
            dfv['hora'] = dfv.index.to_series().apply(lambda x: int(x[1:]))

            dfm = dfh.merge(dfv, on='hora')
            dfm['estacion'] = estacion
            dfm['fecha'] = fecha

            dfn = pd.concat([dfn, dfm])

    dfn[magnitudes_x] = np.where(dfn[magnitudes_y]=='V', dfn[magnitudes_x], np.nan)
    dfn = dfn.rename(columns=dict(zip(magnitudes_x, magnitudes)))
    for m in magnitudes_y:
        del dfn[m]

    dfn.loc[dfn.hora==24, 'fecha'] = dfn.loc[dfn.hora==24, 'fecha'] + pd.DateOffset(days=1)
    dfn.loc[dfn.hora==24, 'hora'] = 0
    dfn['hora'] = dfn.hora.apply(lambda x: f'{x:02}:00:00')
    dfn.fecha = pd.to_datetime(dfn.fecha.astype(str) + " " + dfn.hora)
    del dfn['hora']
    dfn.to_csv(f'{data_path}/02-by-location-month/meteo/{file.split("/")[-1]}', index=False)

In [17]:
def transform_df(df):
    df.loc[df.temperatura<-25] = np.nan
    df.loc[df.humedad_relativa<0] = np.nan

    wv = df.pop('velocidad_viento')
    wd_rad = df.pop('dir_viento')*np.pi / 180
    df['vientox'] = wv*np.cos(wd_rad)
    df['vientoy'] = wv*np.sin(wd_rad)
    
    return df

In [22]:
for j, estacion in enumerate(cods_cortos):
    print(f"{j+1}/{len(cods_cortos)}", end="\r")
    df = pd.DataFrame([])
    for year in years:
        paths = [f'{data_path}/02-by-location-month/meteo/{month_dict[i]}_meteo{year}.csv' for i in range(1, 13)]
        for path in paths:
            dfi = pd.read_csv(path, parse_dates=["fecha"])
            dfi = dfi[dfi.estacion==estacion]
            df = pd.concat([df, dfi])
    df = df.sort_values("fecha")
    df = transform_df(df)
    df.to_csv(f"{data_path}/03-by-location/meteo/estacion-{estacion:.0f}.csv", index=False)